In [1]:
!pip install web3

In [2]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [3]:
POLY_API_KEY = "TYVT9VBDKZUHWS35GYYJP3FK7MTS8XS8K9"
POLY_URL = "https://api.polygonscan.com/api"

web3ProviderURL = "https://rpc.ankr.com/polygon/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(web3ProviderURL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [4]:
topics = [
    "0x783cca1c0412dd0d695e784568c96da2e9c22ff989357a2e8b1d9b2b4e6b7118"
]

In [5]:
factories = {
    "uniswap_v3": {
        "address" : ["0x1f98431c8ad98523631ae4a59f267346ea31f984"],
        "blockInterval" : 1500000,
        },
}

In [6]:
# Get the number of most recent block
def get_latest_block():
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": POLY_API_KEY
    }
    response = requests.post(POLY_URL, params=params)
    data = json.loads(response.text)
    latest_block = int(data['result'], 16)
    return latest_block

In [7]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": POLY_API_KEY
    }
    response = requests.post(POLY_URL, params=params)
    data = json.loads(response.text)
    result = data['result']
    return result

In [8]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [9]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "uniswap_v3" : 10**6,
        "sushiswap_v3" : 10**6,
        "curve" : 10**10,
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    return None


def get_fee_from_swapFeeUnits(contract, name):
    if name == "kyberswap":
        swapFeeUnits = float(contract.functions.swapFeeUnits().call())
        fee = swapFeeUnits / 100000
        return fee
    return None


def get_fee_from_globalState(contract, name):
    if name == "quickswap_v3":
        globalState = contract.functions.globalState().call()
        fee = globalState[2]
        fee = fee / 1000
        return fee
    return None


def get_fee(address, name):
    try:
        contract = get_contract(address)

        fix_fee_dict = {
            "quickswap": 3/1000,
            "sushiswap" : 3/1000,
            "apeswap" : 2/1000,
            "dfyn" : 3/1000,
            "jetswap" : 1/1000,
        }

        fee_fun = [
            get_fee_from_fee,
            get_fee_from_swapFeeUnits,
            get_fee_from_globalState,
        ]

        if name in fix_fee_dict:
            return fix_fee_dict[name]

        for fun in fee_fun:
            fee = fun(contract, name)
            if fee:
                return fee
        return fee
    except Exception:
        return None

In [10]:
def get_tvl(address):
    token_type = ['NATIVE']
    payload = {
        "jsonrpc": "2.0",
        "method": "ankr_getAccountBalance",
        "params": {
            "blockchain": ["polygon"],
            "walletAddress": address,
            "nativeFirst": True,
            "onlyWhitelisted": True,
            "pageSize": 2
        },
        "id": 1
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json"
    }

    response = requests.post(getAccountBalance_url, json=payload, headers=headers)
    time.sleep(1)
    if response.status_code == 200:
        data = json.loads(response.text)
        tvl = data['result']['totalBalanceUsd']
        return tvl
    return None

In [11]:
def get_tokens(address):
    try:
        contract = get_contract(address)

        token0 = contract.functions.token0().call()
        token1 = contract.functions.token1().call()
        
        return token0, token1
    except Exception:
        return None, None

In [12]:
def get_pool_from_events(factory, start_block, end_block):
    type1 = ["quickswap", "sushiswap", "apeswap", "dfyn", "jetswap", "curve"]
    type2 = ["quickswap_v3", "uniswap_v3", "sushiswap_v3", "kyberswap"]

    pool = []
    name = factory
    for factory_address in factories[factory]["address"]:
        for topic in topics:
            params = {
                "module": "logs",
                "action": "getLogs",
                "fromBlock": start_block,
                "toBlock": end_block,
                "address" : factory_address,
                "topic0": topic,
                "apikey": POLY_API_KEY
            }
            response = requests.post(POLY_URL, params=params)
            time.sleep(0.2)
            data = json.loads(response.text)
            logs = data['result']
            for i in tqdm(range(len(logs))):
                log = logs[i]
                hex_string = log['data']
                hex_string = hex_string[2:]
                if factory in type1:
                    address = '0x' + hex_string[24:64]
                elif factory in type2:
                    address = '0x' + hex_string[-40:].lower()
                tvl = get_tvl(address)
                fee = get_fee(address, name)
                token0, token1 = get_tokens(address)
                row = {
                    "name" : name,
                    "factory" : factory_address,
                    "tvl" : tvl,
                    "fee" : fee,
                    "contract" : address,
                    "token0" : token0,
                    "token1" : token1,
                }
                pool.append(row)
    return pool

In [13]:
def to_file(pool, count):
    pool_df = pd.DataFrame(pool)
    pool_df = pool_df.dropna(axis=0).reset_index(drop=True)
    
    file_name = f"part{count}.csv"
    pool_df.to_csv(file_name, index=False)
    return file_name

In [14]:
latest_block = get_latest_block()
latest_block

53017839

In [15]:
files = []
for factory in factories.keys():
    interval = factories[factory]["blockInterval"]
    for count in range(1000):
        start = count * interval
        end = start + interval
        if end > latest_block:
            end = latest_block
        pool = get_pool_from_events(factory, start, end)
        if len(pool) > 0:
            file_name = to_file(pool, count)
            files.append(file_name)
        if end >= latest_block:
            break

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 517/517 [35:52<00:00,  4.16s/it]


In [16]:
combined_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)
    
combined_df = combined_df.drop_duplicates()
combined_df

,name,factory,tvl,fee,contract,token_1,token_2,token0,token1
0,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,425.896468,0.003,0x5d4692de860ad10773467ddb4cc69727dd5171f0,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xdb13a5e277dfb18cf9b8d652761e84e90594e681,NaN,NaN
1,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,0.000214,0.003,0x31022662f0b5821e90ad01613ca124d7c0ab0335,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xaa4765b2734cedbc9d5dd46f8ff6ddb16a28bfac,NaN,NaN
2,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,0.000001,0.003,0xb022f2cc432cf754feb41f53b06ef8e9436066ca,0x1c2806fa5e22a6bd1b3b98a3a5c34ebdcdf02943,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,NaN,NaN
3,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,0.035298,0.003,0x65246daa385cebe1e8bfe381614f8fde595269e4,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619,0x8cb96e95c714e0996e6d299f617f518d965e352e,NaN,NaN
4,quickswap,0x5757371414417b8c6caad45baef941abc7d3ab32,0.000000,0.003,0x83f17fef9512f3cbee745f9ca01f9c928a360d02,0x28b7c90d2cfa1e54bbacd01c030e18a74aeeccf0,0x878534a571ad0f2d4f38bc8a8b2cb393f472580d,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14946,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,96.350901,0.010,0x21215246573e94d992e222506f82e70d5334630b,NaN,NaN,0x50B728D8D964fd00C2d0AAD81718b71311feF68a,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619
14947,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0.434268,0.010,0xc575cbbb8d3a2487d206a730ff33298e65be781b,NaN,NaN,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0x6be5Ae6a7Bad2B07F21FB3Be7F3E48c6dDa96312
14948,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,19520.976923,0.010,0xb25aa3afabdf80664c13819ccb70f77343e2867b,NaN,NaN,0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063,0xc59132FBdF8dE8fbE510F568a5D831C991B4fC38
14949,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,26078.998262,0.010,0x2c44ce8cfc35cb8f6596f4c3e8521294a844e7aa,NaN,NaN,0xc2132D05D31c914a87C6611C10748AEb04B58e8F,0xc59132FBdF8dE8fbE510F568a5D831C991B4fC38


In [17]:
combined_df.to_csv('uniswapv3_combined.csv', index=False)